# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-1. hello-rnn
<hr>
``` python 

# Lab 12 RNN
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hello: hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

y_data = [[1, 0, 2, 3, 3, 4]]    # ihello

input_dim = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6

X = tf.placeholder(
    tf.float32, [None, sequence_length, hidden_size])  # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(t_cnt, "loss:", l, "prediction: ", result, "true Y: ", y_data)

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("\tPrediction str: ", ''.join(result_str))

'''
 1th training... 
  loss : 1.6107723712921143, 	 prediction : [[2 3 3 4 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellooo
 2th training... 
  loss : 1.525847315788269, 	 prediction : [[2 3 3 3 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellloo
 3th training... 
  loss : 1.453248381614685, 	 prediction : [[2 3 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elello
 4th training... 
  loss : 1.381018877029419, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 5th training... 
  loss : 1.3161988258361816, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
  
...
...

 1998th training... 
  loss : 0.6351824998855591, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1999th training... 
  loss : 0.6351807117462158, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 2000th training... 
  loss : 0.635178804397583, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
  
'''

```

<hr>

In [3]:
import tensorflow as tf
import numpy as np

C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
tf.set_random_seed(777)  # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hello: hihell -> ihello
x_data    = [[0, 1, 0, 2, 3, 3]]   # hihell

x_one_hot = [[[1, 0, 0, 0, 0],     # h 0
              [0, 1, 0, 0, 0],     # i 1
              [1, 0, 0, 0, 0],     # h 0
              [0, 0, 1, 0, 0],     # e 2
              [0, 0, 0, 1, 0],     # l 3
              [0, 0, 0, 1, 0]]]    # l 3

y_data = [[1, 0, 2, 3, 3, 4]]      # ihello

In [5]:
input_dim       = 5   # one-hot size
hidden_size     = 5   # output from the LSTM. 5 to directly predict one-hot
batch_size      = 1   # one sentence
sequence_length = 6   # |ihello| == 6


In [6]:
X = tf.placeholder(
    tf.float32, [None, sequence_length, hidden_size])   # X one-hot
Y = tf.placeholder(tf.int32, [None, sequence_length])   # Y label

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X, initial_state=initial_state, dtype=tf.float32)

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [10]:
time1 = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        # print(t_cnt, "loss:", l, "prediction: ", result, "true Y: ", y_data)
        print(" {cnt}th training... \n  loss : {loss}, \t prediction : {pred}, \t true Y : {y_data} ".format(
                cnt  = t_cnt+1, 
                loss = l, 
                pred = result, 
                y_data = y_data
        ), end="\t→ ")

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print("Prediction str :", ''.join(result_str))
        
time2 = time.time()


 1th training... 
  loss : 1.6107723712921143, 	 prediction : [[2 3 3 4 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellooo
 2th training... 
  loss : 1.525847315788269, 	 prediction : [[2 3 3 3 4 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ellloo
 3th training... 
  loss : 1.453248381614685, 	 prediction : [[2 3 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elello
 4th training... 
  loss : 1.381018877029419, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 5th training... 
  loss : 1.3161988258361816, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 6th training... 
  loss : 1.2625374794006348, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 7th training... 
  loss : 1.2124074697494507, 	 prediction : [[2 3 2 3 3 3]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : elelll
 8th training... 
  loss : 1.16801786

  loss : 0.6850883364677429, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 61th training... 
  loss : 0.6838297247886658, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 62th training... 
  loss : 0.682624340057373, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 63th training... 
  loss : 0.6814316511154175, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 64th training... 
  loss : 0.6801770925521851, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 65th training... 
  loss : 0.6788063049316406, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 66th training... 
  loss : 0.6772465109825134, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 67th training... 
  loss : 0.675536572933197, 

  loss : 0.6571952104568481, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 120th training... 
  loss : 0.657023549079895, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 121th training... 
  loss : 0.6568501591682434, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 122th training... 
  loss : 0.6566729545593262, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 123th training... 
  loss : 0.656508207321167, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 124th training... 
  loss : 0.6563576459884644, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 125th training... 
  loss : 0.6562175750732422, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 126th training... 
  loss : 0.65608465671

  loss : 0.6509263515472412, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 178th training... 
  loss : 0.6508530378341675, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 179th training... 
  loss : 0.650780439376831, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 180th training... 
  loss : 0.650709331035614, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 181th training... 
  loss : 0.6506392359733582, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 182th training... 
  loss : 0.6505700349807739, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 183th training... 
  loss : 0.6505018472671509, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 184th training... 
  loss : 0.65043449401

  loss : 0.647727370262146, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 236th training... 
  loss : 0.6476820707321167, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 237th training... 
  loss : 0.6476377248764038, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 238th training... 
  loss : 0.6475951671600342, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 239th training... 
  loss : 0.6475541591644287, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 240th training... 
  loss : 0.6475139856338501, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 241th training... 
  loss : 0.6474733948707581, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 242th training... 
  loss : 0.6474324464

...

  loss : 0.6358652114868164, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1670th training... 
  loss : 0.6358628273010254, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1671th training... 
  loss : 0.6358605623245239, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1672th training... 
  loss : 0.6358581781387329, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1673th training... 
  loss : 0.6358557939529419, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1674th training... 
  loss : 0.6358534097671509, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1675th training... 
  loss : 0.6358511447906494, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1676th training... 
  loss : 0.63

  loss : 0.6356839537620544, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1750th training... 
  loss : 0.6356817483901978, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1751th training... 
  loss : 0.6356796026229858, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1752th training... 
  loss : 0.6356773376464844, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1753th training... 
  loss : 0.6356751918792725, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1754th training... 
  loss : 0.635672926902771, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1755th training... 
  loss : 0.6356707811355591, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1756th training... 
  loss : 0.635

  loss : 0.6355592608451843, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1808th training... 
  loss : 0.6355571746826172, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1809th training... 
  loss : 0.6355551481246948, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1810th training... 
  loss : 0.6355530023574829, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1811th training... 
  loss : 0.6355509161949158, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1812th training... 
  loss : 0.6355487704277039, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1813th training... 
  loss : 0.6355466842651367, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1814th training... 
  loss : 0.63

  loss : 0.6353976130485535, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1887th training... 
  loss : 0.6353956460952759, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1888th training... 
  loss : 0.6353936195373535, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1889th training... 
  loss : 0.6353917121887207, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1890th training... 
  loss : 0.6353896856307983, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1891th training... 
  loss : 0.6353877186775208, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1892th training... 
  loss : 0.6353857517242432, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1893th training... 
  loss : 0.63

  loss : 0.6352443099021912, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1966th training... 
  loss : 0.6352424621582031, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1967th training... 
  loss : 0.6352405548095703, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1968th training... 
  loss : 0.6352386474609375, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1969th training... 
  loss : 0.6352367997169495, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1970th training... 
  loss : 0.6352348923683167, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1971th training... 
  loss : 0.6352330446243286, 	 prediction : [[1 0 2 3 3 4]], 	 true Y : [[1, 0, 2, 3, 3, 4]] 	→ Prediction str : ihello
 1972th training... 
  loss : 0.63

In [9]:
chk_processting_time(time1, time2)

처리시간 : 0분 19초 경과되었습니다.


19.33219027519226

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>